In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import numpy as np
import pandas as pd
import mysql.connector
import logging

In [4]:
DATASET_PATH = '../datasets'

In [5]:
movies      = pd.read_csv(f'{DATASET_PATH}/items.csv')
interactions = pd.read_csv(f'{DATASET_PATH}/interactions.csv')

In [6]:
movies.head(5)

,id,name,description,image
0,1,Toy Story (1995),"Led by Woody, Andy's toys live happily in his ...",https://image.tmdb.org/t/p/w500//uXDfjJbdP4ijW...
1,10,GoldenEye (1995),James Bond must unmask the mysterious head of ...,https://image.tmdb.org/t/p/w500//bFzjdy6ucvNlX...
2,11,"American President, The (1995)","Widowed U.S. president Andrew Shepherd, one of...",https://image.tmdb.org/t/p/w500//yObOAYFIHXHkF...
3,14,Nixon (1995),An all-star cast powers this epic look at Amer...,https://image.tmdb.org/t/p/w500//ORlTkdDwWU94O...
4,15,Cutthroat Island (1995),"Morgan Adams and her slave, William Shaw, are ...",https://image.tmdb.org/t/p/w500//hYdeBZ4BFXivd...


In [7]:
interactions.head(5)

,item_id,user_id,rating
0,1,1,4.0
1,47,1,5.0
2,50,1,5.0
3,70,1,3.0
4,101,1,5.0


In [8]:
connection = mysql.connector.connect(
  host     = "localhost",
  user     = "root",
  password = "1234"
)

In [9]:
def execute(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
    except Exception as error:
        print(error)
        print(f'Query: {query}')


def insert_item(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_item(id, name, description, image)
        VALUES (
            :ID,
            ":NAME",
            ":DESC",
            ":IMAGE"
        );    
        """.replace(':ID', str(row['id'])) \
            .replace(':NAME', str(row['name'])) \
            .replace(':DESC', str(row['description']).replace('"', '')) \
            .replace(':IMAGE', str(row['image']))
    execute(connection, query)
        
def insert_interaction(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_interaction(item_id, user_id, rating)
        VALUES (
            ":ITEM_ID",
            ":USER_ID",
            ":RATING"
        );    
        """.replace(':ITEM_ID', str(int(row['item_id']))) \
            .replace(':USER_ID', str(int(row['user_id']))) \
            .replace(':RATING', str(row['rating']))
    execute(connection, query)

In [10]:
execute(connection, 'ALTER TABLE recsys.recsysweb_item MODIFY id int(11) NOT NULL')
execute(connection, 'DELETE FROM recsys.recsysweb_item')

In [11]:
for _, row in movies.iterrows():
    insert_item(connection, row)    

In [12]:
execute(connection, 'DELETE FROM recsys.recsysweb_interaction')

In [13]:
for _, row in interactions.iterrows():
    insert_interaction(connection, row)

In [14]:
connection.close()

In [15]:
interactions.user_id.max()

610